In [ ]:
# installation du package de la pipeline pour utiliser
# le code d'extraction de l'annuaire du service public

!pip install -e ../../../pipeline

In [ ]:
import elasticsearch as es
import pandas as pd


Ce notebook illustre la siretisation d'un jeu de données grâce à l'indexation de la base sirene dans elasticsearch.

Il faut préalablement avoir (ré)exécuté le dag d'import de la base sirene en local.

Le jeu de données est l'ensemble des mairies de l'annuaire du service public, qui est déjà correctement sirétisé, afin d'évaluer l'heuristique de sirétisation.


In [ ]:


es_conn = es.Elasticsearch("http://localhost:9200")  # utilisation de l'index local


In [ ]:
def find_matches(
    nom: str,
    adresse: str,
    latitude: float,
    longitude: float,
    commune: str,
):
    query = {
        "bool": {
            "filter": [],
            "should": [],
            "must": [],
        }
    }

    # query["bool"]["should"].append({"match": {"nom_complet": nom}})
    # query["bool"]["should"].append({"match": {"adresse_complete": adresse}})
    query["bool"]["should"].append(
        {"match": {"libelle_commune_etablissement": commune}}
    )
    query["bool"]["filter"].append(
        {"match": {"activite_principale_etablissement": "84.11Z"}}
    )
    query["bool"]["must"].append(
        {
            "geo_distance": {
                "distance": "10km",
                "coordonnees": {"lat": latitude, "lon": longitude},
            }
        }
    )
    query["bool"]["filter"].append(
        {
            "bool": {
                "should": [
                    {"match": {"categorie_juridique_unite_legale": 7210}},
                    {"match": {"categorie_juridique_unite_legale": 7312}},
                ]
            }
        }
    )
    query["bool"]["should"].append(
        {"match": {"categorie_juridique_unite_legale": 7210}}
    )
    query["bool"]["filter"].append({"match": {"etablissement_siege": True}})

    response = es_conn.search(index="siret", query=query, size=10)
    return (
        next(iter(response["hits"]["hits"]), None)
        if "hits" in response and "hits" in response["hits"]
        else None
    )


In [ ]:
from pathlib import Path
from data_inclusion.scripts.tasks import annuaire_du_service_public

df = annuaire_du_service_public.read(Path("/home/vmttn/Downloads/all_latest.tar.bz2"))

In [ ]:
df1 = df[["id", "siret", "nom", "ancien_code_pivot", "adresse"]]
df1 = df1[df.ancien_code_pivot.str.contains("mairie")]
df1 = df1[["id", "siret", "nom"]].merge(
    pd.json_normalize(
        df1.to_dict(orient="records"), record_path="adresse", meta="id"
    ).drop_duplicates("id")[["id", "numero_voie", "nom_commune", "latitude", "longitude"]],
    on="id",
)


In [ ]:
df2 = df1.sample(1000)

In [ ]:
df3 = df2.merge(
    df2.apply(
        lambda row: find_matches(
            row.nom,
            row.numero_voie,
            row.latitude,
            row.longitude,
            row.nom_commune,
        ),
        axis=1,
    ).apply(pd.Series),
    left_index=True,
    right_index=True,
)


In [ ]:
pd.set_option('display.max_rows', 500)

# comparaison avec les résultats de la siretisation automatique

df3[df3.siret != df3._id][["siret", "_id", "_score", "nom_commune", "nom"]].sort_values("_score", ascending=False)